In [1]:
import pandas as pd 
import nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import string 
import re

In [2]:
data_raw = pd.read_csv("tweets_uuite_TA.csv")
data_raw

,Datetime,Tweet Id,Tweet,Username
0,2021-02-19 23:58:25+00:00,1362914466917011456,"Dibeberkan Roy Suryo, Ternyata Ini Sejarah Lah...",idtodayco
1,2021-02-19 23:55:29+00:00,1362913729155764224,@TsamaraDKI @psi_id Demi menyelamatkan seorang...,toha_laha
2,2021-02-19 23:54:18+00:00,1362913430143836162,"Makassar Terkini: Jokowi Minta Revisi UU ITE, ...",bryanesia
3,2021-02-19 23:43:48+00:00,1362910790777331719,UU ITE dibuat pada zaman pra sejarah ??? Pada ...,amalpribadi2
4,2021-02-19 23:40:19+00:00,1362909911743520771,Pada UU ITE pasal-pasal karet yang perlu dicab...,BournAgainShell
...,...,...,...,...
11781,2021-02-14 00:08:58+00:00,1360742794487209989,@msaid_didu @Pak_JK @kiangiekwik @m_dinsyamsud...,Toraya_Maelo
11782,2021-02-14 00:06:15+00:00,1360742111662940162,Kalo di Jepang 4 orang terluka.kalo di Indones...,mugiarso74
11783,2021-02-14 00:03:22+00:00,1360741387050897410,"@adityapradanan Wkwk ntaps lah ditegasin, uran...",jmkaisa97
11784,2021-02-14 00:01:31+00:00,1360740920472178694,"Demokrat: Jika Jokowi Minta Dikritik, UU ITE H...",GafIkhsan


In [3]:
nospam=data_raw[data_raw.groupby('Username').Username.transform('count')<21].copy()
nospam

,Datetime,Tweet Id,Tweet,Username
1,2021-02-19 23:55:29+00:00,1362913729155764224,@TsamaraDKI @psi_id Demi menyelamatkan seorang...,toha_laha
2,2021-02-19 23:54:18+00:00,1362913430143836162,"Makassar Terkini: Jokowi Minta Revisi UU ITE, ...",bryanesia
3,2021-02-19 23:43:48+00:00,1362910790777331719,UU ITE dibuat pada zaman pra sejarah ??? Pada ...,amalpribadi2
4,2021-02-19 23:40:19+00:00,1362909911743520771,Pada UU ITE pasal-pasal karet yang perlu dicab...,BournAgainShell
5,2021-02-19 23:37:04+00:00,1362909094118522882,@PutraWadapi Apakah revisi UU ITE semakin baik...,yanzulfian9
...,...,...,...,...
11781,2021-02-14 00:08:58+00:00,1360742794487209989,@msaid_didu @Pak_JK @kiangiekwik @m_dinsyamsud...,Toraya_Maelo
11782,2021-02-14 00:06:15+00:00,1360742111662940162,Kalo di Jepang 4 orang terluka.kalo di Indones...,mugiarso74
11783,2021-02-14 00:03:22+00:00,1360741387050897410,"@adityapradanan Wkwk ntaps lah ditegasin, uran...",jmkaisa97
11784,2021-02-14 00:01:31+00:00,1360740920472178694,"Demokrat: Jika Jokowi Minta Dikritik, UU ITE H...",GafIkhsan


In [4]:
# data=nospam.sample(n=int(len(nospam)*90/100))
# data


In [5]:
username = nospam.copy()
username.drop(["Datetime", "Tweet Id", "Tweet"], axis = 1)


,Username
1,toha_laha
2,bryanesia
3,amalpribadi2
4,BournAgainShell
5,yanzulfian9
...,...
11781,Toraya_Maelo
11782,mugiarso74
11783,jmkaisa97
11784,GafIkhsan


In [6]:
uniq_username=""
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 30):
    uniq_username=username['Username'].str.split(expand=True).stack().value_counts()
    
len(uniq_username)

5945

## Case Folding
Mengubah semua huruf menjadi huruf kecil (lower case) (library pandas)

In [7]:
dataclean = nospam.drop(["Datetime", "Tweet Id", "Username"], axis = 1)
dataclean.columns = dataclean.columns.str.strip().str.lower()
dataclean.head(10)

,tweet
1,@TsamaraDKI @psi_id Demi menyelamatkan seorang...
2,"Makassar Terkini: Jokowi Minta Revisi UU ITE, ..."
3,UU ITE dibuat pada zaman pra sejarah ??? Pada ...
4,Pada UU ITE pasal-pasal karet yang perlu dicab...
5,@PutraWadapi Apakah revisi UU ITE semakin baik...
6,UU ITE itu UU No 11 Tahin 2008.\nTahun 2008 si...
8,@taharuddin_id Kalau dibandingin di negeri ini...
10,"Anggota Partainya Suka Bikin Laporan, Tsamara ..."
11,Wacana Revisi UU ITE Diendus sebagai Upaya Bar...
12,@keuangannews_id @budimandjatmiko benar enggak...


In [8]:
dataclean['tweet'] = dataclean['tweet'].str.lower()
print('Hasil Case Folding : \n')
print(dataclean['tweet'].head(10))
print('\n\n\n')

Hasil Case Folding : 

1     @tsamaradki @psi_id demi menyelamatkan seorang...
2     makassar terkini: jokowi minta revisi uu ite, ...
3     uu ite dibuat pada zaman pra sejarah ??? pada ...
4     pada uu ite pasal-pasal karet yang perlu dicab...
5     @putrawadapi apakah revisi uu ite semakin baik...
6     uu ite itu uu no 11 tahin 2008.\ntahun 2008 si...
8     @taharuddin_id kalau dibandingin di negeri ini...
10    anggota partainya suka bikin laporan, tsamara ...
11    wacana revisi uu ite diendus sebagai upaya bar...
12    @keuangannews_id @budimandjatmiko benar enggak...
Name: tweet, dtype: object






In [9]:
import string 
import re

# CLEANSING
Menghapus seluruh tanda baca, link, hashtag, mention

In [10]:
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    #remove url (menghapus link)
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)(?:(?:\/[^\s/]))*', '', text)
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
dataclean['tweet'] = dataclean['tweet'].apply(remove_tweet_special)

#remove number (menghapus angka)
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataclean['tweet'] = dataclean['tweet'].apply(remove_number)

#remove punctuation (menghapus tanda baca)
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataclean['tweet'] = dataclean['tweet'].apply(remove_punctuation)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataclean['tweet'] = dataclean['tweet'].apply(remove_singl_char)

dataclean.head()

#remove whitespace leading & trailing (menghapus spasi awal dan akhir)
def remove_whitespace_LT(text):
    return text.strip()

dataclean['tweet'] = dataclean['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataclean['tweet'] = dataclean['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataclean['tweet'] = dataclean['tweet'].apply(remove_singl_char)

dataclean

,tweet
1,id demi menyelamatkan seorang abu janda agar t...
2,makassar terkini jokowi minta revisi uu ite ro...
3,uu ite dibuat pada zaman pra sejarah pada saat...
4,pada uu ite pasalpasal karet yang perlu dicabu...
5,apakah revisi uu ite semakin baik atau semakin...
...,...
11781,didu jk dinsyamsuddin gak bisa bedakan kritikh...
11782,kalo di jepang orang terlukakalo di indonesia ...
11783,wkwk ntaps lah ditegasin urang males kalo gili...
11784,demokrat jika jokowi minta dikritik uu ite har...


In [11]:
dataset=dataclean

In [12]:
# bruto = int(len(dataset))
# dataset = dataset.drop_duplicates(subset=['tweet'])
# print("Dataset dibuang (Karena duplikat) : "+(str(bruto-int(len(dataset))))+" data")
# print("Dataset masuk : "+str(len(dataset))+" data")

## TOKENIZING
membagi kalimat menjadi perkata

In [13]:
nltk.download('punkt')
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataset['tweet'] = dataset['tweet'].apply(word_tokenize_wrapper)
dataset['tweet']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1        [id, demi, menyelamatkan, seorang, abu, janda,...
2        [makassar, terkini, jokowi, minta, revisi, uu,...
3        [uu, ite, dibuat, pada, zaman, pra, sejarah, p...
4        [pada, uu, ite, pasalpasal, karet, yang, perlu...
5        [apakah, revisi, uu, ite, semakin, baik, atau,...
                               ...                        
11781    [didu, jk, dinsyamsuddin, gak, bisa, bedakan, ...
11782    [kalo, di, jepang, orang, terlukakalo, di, ind...
11783    [wkwk, ntaps, lah, ditegasin, urang, males, ka...
11784    [demokrat, jika, jokowi, minta, dikritik, uu, ...
11785    [andalan, rejim, ini, adalah, uu, ite, buzzerp...
Name: tweet, Length: 10041, dtype: object

## NORMALIZE
MENGUBAH KATA TIDAK BAKU MENJADI KATA BAKU

In [14]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataset['tweet'] = dataset['tweet'].apply(normalized_term)
dataset['tweet'].head(10)

1     [indonesia, demi, menyelamatkan, seorang, abu,...
2     [makassar, terkini, jokowi, minta, revisi, uu,...
3     [uu, ite, dibuat, pada, zaman, pra, sejarah, p...
4     [pada, uu, ite, pasal pasal, karet, yang, perl...
5     [apakah, revisi, uu, ite, semakin, baik, atau,...
6     [uu, ite, itu, uu, no, tahin, tahun, siapa, co...
8     [indonesia, kalau, dibandingin, di, negeri, in...
10    [anggota, partainya, suka, bikin, laporan, tsa...
11    [wacana, revisi, uu, ite, diendus, sebagai, up...
12    [indonesia, benar, tidak, mas, saya, tidak, pe...
Name: tweet, dtype: object

## STOPWORD REMOVAL
Membuang kata yang tidak memiliki arti penting

In [15]:
from nltk.corpus import stopwords
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
list_stopwords.extend(["amp", "nya"])
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)
# list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
list_stopwords = set(list_stopwords)
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]
dataset['tweet'] = dataset['tweet'].apply(stopwords_removal) 
dataset.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet
1,"[indonesia, menyelamatkan, abu, janda, terjera..."
2,"[makassar, terkini, jokowi, revisi, uu, ite, r..."
3,"[uu, ite, zaman, pra, sejarah, orang, pintar, ..."
4,"[uu, ite, pasal pasal, karet, dicabut, pasal, ..."
5,"[revisi, uu, ite, modus, khawatir, revisi, buz..."


In [16]:
dataset.to_csv('dataclean_uu_ite.csv', index=False)
dataclean = pd.read_csv('dataclean_uu_ite.csv', encoding='latin1')
dataclean

,tweet
0,"['indonesia', 'menyelamatkan', 'abu', 'janda',..."
1,"['makassar', 'terkini', 'jokowi', 'revisi', 'u..."
2,"['uu', 'ite', 'zaman', 'pra', 'sejarah', 'oran..."
3,"['uu', 'ite', 'pasal pasal', 'karet', 'dicabut..."
4,"['revisi', 'uu', 'ite', 'modus', 'khawatir', '..."
...,...
10036,"['didu', 'jk', 'dinsyamsuddin', 'bedakan', 'kr..."
10037,"['jepang', 'orang', 'terlukakalo', 'indonesia'..."
10038,"['wkwk', 'ntaps', 'ditegasin', 'males', 'gilir..."
10039,"['demokrat', 'jokowi', 'dikritik', 'uu', 'ite'..."


In [17]:
list_stopwords

{'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'amp',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 '